Databricks es un servicio para el análisis de grandes volúmenes de datos mediante el uso del framework de computación distribuida Apache Spark. En su versión Community, ofrece la posibilidad de levantar de forma gratuita un cluster de 15,25GB de memoria RAM y 2 cores, que nos permite ejecutar código Spark.

In [1]:
# encoding: utf-8

<span style="text-decoration: underline">**Caso práctico de iniciación a PySpark**:</span>
  - Se selecciona una muestra de 30 clientes de una compañía de telecomunicaciones.
  - Tenemos 5 tablas de entrada en las que aparecen representados estos 30 clientes:
    -  **clientes (df_clientes)**: muestra informacion de cada cliente (id,nombre,edad,ciudad,pais)
    -  **factura_mes_ant (df_facturas_mes_ant)**: Muestra la factura del mes anterior junto con el plan que tenían contratado para cada cliente. Incluye un campo que determina si el precio de la oferta es estándar o representa una oferta especial.
    -  **factura_mes_actual (df_facturas_mes_actual)**: muestra la factura acumulada por cada cliente al final del mes actual.
    -  **consumos_diarios (df_consumos_diarios)**: muestra para cada cliente y diariamente, los datos móviles consumidos, el tráfico, SMS enviados, minutos llamadas a fijos y moviles.
    -  **ofertas_estandar (df_ofertas)**: muestra un catalogo de los distintos paquetes ofrecidos por la empresa, junto con una descripción y sus precios. <br>
  - Se irán pidiendo diferentes casuísticas mediante las cuáles nos iremos familiarizando con los diferenes comandos de Spark en Python </br>
  ***Nota: Todos los valores que hacen referencia a un precio están expresados en euros (€), independientemente del país del que proceda el cliente***

En primer lugar debería crearse una sesión de Spark ejecutando los comandos siguientes. Sin embargo, Databricks la crea automáticamente por lo que este paso no es necesario.

In [2]:
# Símbolo asterisco para crear comentarios que no afectan a la hora de ejecutar una celda con código PySpark
import pandas as pd
import findspark
findspark.init() #inicializamos
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("PySpark_training").getOrCreate() 
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName('abc').getOrCreate()

**Importación de librerías**

**Creación DataFrames:** 
- Creamos por un lado una variable lista o secuencia que contiene las filas del DataFrame. Cada elemento de esta lista es una tupla que se corresponde con una fila del DataFrame o tabla a crear. 
- Creamos otra variable lista que contiene todas las columnas del DataFrame.
- Para crear el DataFrame utilizamos el comando "spark.createDataFrame" al cual pasamos 2 argumentos: los datos y las columnas comentadas previamente

**DataFrame clientes**

In [4]:
columns = ['id_cliente','nombre','edad','sexo','provincia','pais']
data = [(1,'Pablo Perez',26,'M','Madrid','España'),(2,'Eduardo Redondo',58,'M','Bogota','Colombia'),(3,'Roberto Salazar',68,'M','Monterrey','Mexico'),
        (4,'Pedro Conde',32,'M','Madrid','España'),(5,'Ana Robles',41,'F','Valladolid','España'),(6,'David Roldan',74,'M','Guadalajara','Mexico'),
        (7,'Carmen Arauzo',19,'F','Medellin','Colombia'),(8,'Silvia Rodriguez',49,'F','Barcelona','España'),(9,'David Cardoso',24,'M','Cali','Colombia'),
        (10,'Pablo Lopez',40,'M','Sevilla','España'),(11,'Luis Rodrigues',20,'M','Bogota','Colombia'),(12,'Alvaro Monroy',75,'M','Monterrey','Mexico'),
        (13,'Victor Ruiz',53,'M','Valladolid','España'),(14,'Melisa Aguado',30,'F','Bogota','Colombia'),(15,'Cristian Cuadrado',52,'M','Guadalajara','Mexico'),
        (16,'Cristina Sanz',49,'F','Madrid','España'),(17,'Jorge Recio',18,'M','Valladolid','España'),(18,'Laura Luiz',44,'F','Cali','Colombia'),
        (19,'Juan Carlos Iglesias',38,'M','Barcelona','España'),(20,'Oscar Rico',22,'M','Valladolid','España'),(21,'Fatima Cuevas',29,'F','Cali','Colombia'),
        (22,'Clara Suarez',21,'F','Sevilla','España'),(23,'Fernanda Gomez',78,'F','Monterrey','Mexico'),(24,'Ruben Garcia',68,'M','Sevilla','España'),
        (25,'Ines Barcero',29,'F','Guadalajara','Mexico'),(26,'Celia Castro',47,'F','Barcelona','España'),(27,'Roberta Varado',64,'F','Cali','Colombia'),
        (28,'Walter Ramos',54,'M','Cali','Colombia'),(29,'Romina Verde',29,'F','Guadalajara','Mexico'),(30,'Marta Rodrigo',67,'F','Guadalajara','Mexico')]
        
        #(25,'Roberto Versero',66,'Sevilla','M','España'),(26,'Javier Ramos',36,'M','Madrid','España'),(27,'Ana Bolana',32,'F','Barcelona','España'),
        #(28,'Lucia Alonso',23,'F','Valladolid','España'),(29,'Elena Luengo',27,'F','Valencia','España'),(30,'Rebeca Velez',69,'F','Cali','Colombia'),]

df_clientes = spark.createDataFrame(data, columns)
#df_clientes.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_clientes')
#df_clientes.show(n=30)





**DataFrame Ofertas estandar** (truncate=False)

In [5]:
# Se presentan 10 tipos de ofertas diferentes (combinaciones entre Fibra Optica, Fijo, Movil (llamadas o mensajes ilimitados), Television)

columns = ['id_oferta','descripcion','importe'] 
data = [(1,'Fibra Optica 300MB + Fijo ilimitado + Television',70.00),
        (2,'Fibra Optica 600MB + Fijo ilimitado + Television', 78.50),
        (3,'Fibra Optica 300MB + Movil llamadas y datos ilimitados + Television', 85.00),
        (4,'Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television', 95.00),
        (5,'Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados', 102.50),
        (6,'Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television', 124.50),
        (7,'Fibra Optica 300MB + Movil llamadas ilimitadas + Television', 80.50),
        (8,'Fibra Optica 300MB + Movil datos ilimitados + Television', 84.00),
        (9,'Fijo llamadas ilimitadas + Movil llamadas y datos ilimitados', 42.50),
        (10,'Fijo llamadas ilimitadas + Movil llamadas ilimitadas', 31.99)]

df_ofertas = spark.createDataFrame(data, columns)

#display(df_ofertas)
#df_ofertas.show(n=10, truncate=False)
#df_ofertas.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_ofertas')
#df_ofertas.show()


In [9]:
columns = ['id_cliente','id_oferta','importe','fecha'] 
data = [(1,6,124.5,"2020-07-31"),(1,6,118.60,"2020-07-31"),(1,9,36.5,"2020-07-31"),
        (2,4,95.0,"2020-07-31"),(3,5,102.5,"2020-07-31"),(5,7,80.5,"2020-07-31"),
        (5,9,40.5,"2020-07-31"),(6,8,84.0,"2020-07-31"),(7,4,95.0,"2020-07-31"),
        (8,5,102.5,"2020-07-31"),(10,6,124.5,"2020-07-31"),(11,3,80.0,"2020-07-31"),
        (12,1,70.0,"2020-07-31"),(13,4,95.0,"2020-07-31"),(14,9,42.5,"2020-07-31"),
        (15,2,78.50,"2020-07-31"),(15,6,115.0,"2020-07-31"),(16,1,70.0,"2020-07-31"),
        (17,8,84.0,"2020-07-31"),(18,5,102.5,"2020-07-31"),(18,9,37.5,"2020-07-31"),
        (20,4,95.0,"2020-07-31"),(22,2,78.5,"2020-07-31"),(22,7,69.5,"2020-07-31"),
        (23,3,85.0,"2020-07-31"),(24,3,85.0,"2020-07-31"),(25,9,42.5,"2020-07-31"),
        (25,10,31.99,"2020-07-31"),(26,4,95.0,"2020-07-31"),(27,8,84.0,"2020-07-31"),
        (27,9,41.5,"2020-07-31"),(29,6,124.5,"2020-07-31"),(30,1,70.0,"2020-07-31"),
        (30,2,70.0,"2020-07-31")]
        
df_facturas_mes_ant = spark.createDataFrame(data, columns)
display(df_facturas_mes_ant)
# df_facturas_mes_ant.show(n=40, truncate=False) #si el string es muy largo trunca los caracteres
# df_facturas_mes_ant.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_facturas_mes_ant')
# df_facturas_mes_ant.show()

DataFrame[id_cliente: bigint, id_oferta: bigint, importe: double, fecha: string]

In [10]:
#df_ofertas.filter(df_ofertas.descripcion.contains("300MB")).show()
#df_join = df_facturas_mes_ant.join(df_ofertas, ["id_oferta"],"left").orderBy("id_oferta").withColumn("fibra_300MB",when(col("descripcion").contains("300MB"),lit(1)))
#df_join = df_join.withColumn("movil_llamadas_ilimitadas",when(col("descripcion").contains("llamadas ilimitadas"),lit(1)))
#df_join = df_join.withColumn("televisión",when(col("descripcion").contains("Television"),lit(1)))
#df_join.na.fill(value=0,subset=["fibra_300MB","movil_llamadas_ilimitadas","televisión"]).show()

**DataFrame factura_mes_actual**

In [11]:
# Se dan de baja en el mes actual: 1y,27(solo de uno de los contratos),16,20,24
# cliente 25 rebaja por fidelidad
columns = ['id_cliente','id_oferta','importe','fecha'] 
data = [(1,6,124.5,"2020-08-31"),(1,6,118.60,"2020-08-31"),(2,4,95.0,"2020-08-31"),
        (3,5,102.5,"2020-08-31"),(4,1,70.0,"2020-08-31"),(5,7,80.5,"2020-08-31"),
        (5,9,40.5,"2020-08-31"),(6,8,84.0,"2020-08-31"),(7,4,95.0,"2020-08-31"),
        (8,5,100.5,"2020-08-31"),(9,7,78.50,"2020-08-31"),(9,10,29.99,"2020-08-31"),
        (10,6,124.5,"2020-08-31"),(11,3,83.0,"2020-08-31"),(12,1,70.0,"2020-08-31"),
        (13,4,95.0,"2020-08-31"),(14,9,42.5,"2020-08-31"),(15,2,78.50,"2020-08-31"),
        (15,6,115.0,"2020-08-31"),(17,8,84.0,"2020-08-31"),(18,5,102.5,"2020-08-31"),
        (18,9,39.0,"2020-08-31"),(19,3,85.0,"2020-08-31"),(21,6,124.50,"2020-08-31"),
        (22,2,78.5,"2020-08-31"),(22,7,72.5,"2020-08-31"),(23,3,85.0,"2020-08-31"),
        (25,9,42.5,"2020-08-31"),(25,10,29.70,"2020-08-31"),(26,4,95.0,"2020-08-31"),
        (27,8,84.0,"2020-08-31"),(28,5,102.50,"2020-08-31"),(29,6,121.50,"2020-08-31"),
        (30,1,70.0,"2020-08-31"),(30,2,71.0,"2020-08-31")]

df_facturas_mes_actual = spark.createDataFrame(data, columns)
#display(df_facturas_mes_actual)
#df_facturas_mes_actual.show(n=40, truncate=False)
# df_facturas_mes_actual.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_facturas_mes_actual')

**DataFrame consumos_diarios** (guardar DF en Filesystem Databricks DBFS o generar un CSV a partir del DF)

In [12]:
columns = ['id_cliente','consumo_datos_MB','sms_enviados','minutos_llamadas_movil','minutos_llamadas_fijo','fecha']
data = [(1,664,3,25,0,"2020-08-01"),(1,943,0,12,5,"2020-08-02"),(1,1035,1,46,10,"2020-08-03"),(1,760,3,17,0,"2020-08-04"),
        (1,1409,1,31,4,"2020-08-05"),(1,1627,0,20,0,"2020-08-06"),(1,739,3,21,0,"2020-08-07"),(1,760,0,12,0,"2020-08-08"),
        (1,928,0,0,24,"2020-08-09"),(1,829,2,46,0,"2020-08-10"),(1,1810,3,0,0,"2020-08-11"),(1,760,0,25,0,"2020-08-12"),
        (1,603,0,41,0,"2020-08-13"),(1,732,0,34,102,"2020-08-14"),(1,729,1,0,50,"2020-08-15"),(1,760,1,31,0,"2020-08-16"),
        (1,1934,2,78,0,"2020-08-17"),(1,827,0,29,32,"2020-08-18"),(1,691,0,0,7,"2020-08-19"),(1,760,2,19,0,"2020-08-20"),
        (1,1142,5,14,0,"2020-08-21"),(1,1242,1,40,0,"2020-08-22"),(1,830,0,32,2,"2020-08-23"),(1,760,4,56,17,"2020-08-24"),
        (1,734,0,19,0,"2020-08-25"),(1,1729,0,11,0,"2020-08-26"),(1,819,1,95,18,"2020-08-27"),(1,760,0,60,12,"2020-08-28"),
        (1,302,0,29,0,"2020-08-29"),(1,497,3,14,0,"2020-08-30"),(1,1281,2,21,0,"2020-08-31"),
        (2,321,7,38,13,"2020-08-01"),(2,145,0,8,0,"2020-08-02"),(2,420,1,46,4,"2020-08-03"),(2,492,1,9,0,"2020-08-04"),
        (2,264,8,29,31,"2020-08-05"),(2,172,10,22,0,"2020-08-06"),(2,314,2,74,34,"2020-08-07"),(2,312,2,0,16,"2020-08-08"),
        (2,312,11,32,27,"2020-08-09"),(2,243,2,48,0,"2020-08-10"),(2,423,4,38,12,"2020-08-11"),(2,492,1,10,22,"2020-08-12"),
        (2,492,12,29,29,"2020-08-13"),(2,274,4,60,0,"2020-08-14"),(2,402,0,28,20,"2020-08-15"),(2,471,0,12,30,"2020-08-16"),
        (2,462,1,36,32,"2020-08-17"),(2,294,3,78,42,"2020-08-18"),(2,593,0,29,65,"2020-08-19"),(2,380,12,53,9,"2020-08-20"),
        (2,568,2,42,20,"2020-08-21"),(2,300,2,40,35,"2020-08-22"),(2,600,0,35,29,"2020-08-23"),(2,397,9,37,40,"2020-08-24"),
        (2,623,4,59,42,"2020-08-25"),(2,302,1,34,15,"2020-08-26"),(2,319,1,52,34,"2020-08-27"),(2,901,8,28,83,"2020-08-28"),
        (2,611,0,41,38,"2020-08-29"),(2,319,0,21,19,"2020-08-30"),(2,387,3,19,18,"2020-08-31"),
        (3,123,5,42,14,"2020-08-01"),(3,460,4,24,16,"2020-08-02"),(3,291,3,27,18,"2020-08-03"),(3,149,7,20,0,"2020-08-04"),
        (3,182,6,20,19,"2020-08-05"),(3,245,12,28,21,"2020-08-06"),(3,302,2,31,26,"2020-08-07"),(3,200,8,18,8,"2020-08-08"),
        (3,239,7,40,25,"2020-08-09"),(3,111,9,29,0,"2020-08-10"),(3,191,1,14,29,"2020-08-11"),(3,28,12,4,10,"2020-08-12"),
        (3,201,3,15,32,"2020-08-13"),(3,102,8,31,0,"2020-08-14"),(3,62,10,29,31,"2020-08-15"),(3,82,2,49,42,"2020-08-16"),
        (3,72,1,0,15,"2020-08-17"),(3,94,2,12,18,"2020-08-18"),(3,30,11,20,45,"2020-08-19"),(3,172,1,63,8,"2020-08-20"),
        (3,80,1,12,30,"2020-08-21"),(3,274,7,5,11,"2020-08-22"),(3,219,7,8,15,"2020-08-23"),(3,104,7,6,2,"2020-08-24"),
        (3,180,9,48,45,"2020-08-25"),(3,210,0,10,44,"2020-08-26"),(3,310,2,17,18,"2020-08-27"),(3,182,9,0,1,"2020-08-28"),
        (3,164,0,32,5,"2020-08-29"),(3,289,0,12,33,"2020-08-30"),(3,126,3,23,22,"2020-08-31"),
        (4,523,0,87,0,"2020-08-01"),(4,628,1,56,0,"2020-08-02"),(4,1021,0,15,0,"2020-08-03"),(4,610,1,45,0,"2020-08-04"),
        (4,612,0,65,0,"2020-08-05"),(4,359,0,102,15,"2020-08-06"),(4,832,1,62,0,"2020-08-07"),(4,583,0,30,0,"2020-08-08"),
        (4,490,2,81,20,"2020-08-09"),(4,479,0,81,0,"2020-08-10"),(4,792,2,19,0,"2020-08-11"),(4,1216,0,27,32,"2020-08-12"),
        (4,512,0,32,0,"2020-08-13"),(4,824,4,76,0,"2020-08-14"),(4,682,1,37,18,"2020-08-15"),(4,829,2,47,0,"2020-08-16"),
        (4,921,0,45,0,"2020-08-17"),(4,729,0,42,0,"2020-08-18"),(4,534,2,95,64,"2020-08-19"),(4,967,3,51,82,"2020-08-20"),
        (4,721,3,54,0,"2020-08-21"),(4,218,0,38,17,"2020-08-22"),(4,729,0,85,0,"2020-08-23"),(4,826,1,38,0,"2020-08-24"),
        (4,845,1,34,0,"2020-08-25"),(4,367,2,71,21,"2020-08-26"),(4,502,0,60,0,"2020-08-27"),(4,800,0,25,0,"2020-08-28"),
        (4,629,0,49,0,"2020-08-29"),(4,588,0,50,0,"2020-08-30"),(4,693,1,31,40,"2020-08-31"),
        (5,241,1,125,20,"2020-08-01"),(5,315,0,36,32,"2020-08-02"),(5,301,1,20,80,"2020-08-03"),(5,152,5,38,62,"2020-08-04"),
        (5,289,4,110,45,"2020-08-05"),(5,510,2,74,40,"2020-08-06"),(5,178,2,19,67,"2020-08-07"),(5,241,3,41,59,"2020-08-08"),
        (5,301,6,62,39,"2020-08-09"),(5,86,4,82,47,"2020-08-10"),(5,201,6,75,61,"2020-08-11"),(5,358,8,51,62,"2020-08-12"),
        (5,176,8,45,0,"2020-08-13"),(5,109,5,102,12,"2020-08-14"),(5,798,5,91,45,"2020-08-15"),(5,301,0,43,71,"2020-08-16"),
        (5,198,9,73,70,"2020-08-17"),(5,125,2,115,17,"2020-08-18"),(5,182,4,99,49,"2020-08-19"),(5,222,0,30,80,"2020-08-20"),
        (5,234,3,82,34,"2020-08-21"),(5,201,3,70,20,"2020-08-22"),(5,283,3,130,27,"2020-08-23"),(5,333,2,28,0,"2020-08-24"),
        (5,387,5,90,41,"2020-08-25"),(5,258,5,62,30,"2020-08-26"),(5,251,7,100,36,"2020-08-27"),(5,475,6,79,16,"2020-08-28"),
        (5,312,12,15,80,"2020-08-29"),(5,328,1,50,19,"2020-08-30"),(5,291,11,32,65,"2020-08-31"),
        (6,50,7,10,50,"2020-08-01"),(6,80,6,10,0,"2020-08-02"),(6,65,12,0,40,"2020-08-03"),(6,284,14,0,42,"2020-08-04"),
        (6,20,9,0,46,"2020-08-05"),(6,36,4,0,67,"2020-08-06"),(6,90,9,10,36,"2020-08-07"),(6,201,18,0,83,"2020-08-08"),
        (6,28,13,0,64,"2020-08-09"),(6,150,7,22,46,"2020-08-10"),(6,135,7,0,65,"2020-08-11"),(6,196,22,17,68,"2020-08-12"),
        (6,49,15,0,37,"2020-08-13"),(6,0,0,0,92,"2020-08-14"),(6,122,6,0,56,"2020-08-15"),(6,146,3,0,94,"2020-08-16"),
        (6,102,13,15,48,"2020-08-17"),(6,38,8,0,17,"2020-08-18"),(6,180,11,14,41,"2020-08-19"),(6,159,6,0,26,"2020-08-20"),
        (6,27,16,24,50,"2020-08-21"),(6,156,12,0,75,"2020-08-22"),(6,193,10,20,39,"2020-08-23"),(6,478,4,30,64,"2020-08-24"),
        (6,35,5,0,80,"2020-08-25"),(6,120,15,28,52,"2020-08-26"),(6,201,8,0,54,"2020-08-27"),(6,310,1,28,15,"2020-08-28"),
        (6,40,3,8,35,"2020-08-29"),(6,73,4,16,34,"2020-08-30"),(6,174,7,25,13,"2020-08-31"),
        (7,1456,0,37,0,"2020-08-01"),(7,946,0,24,24,"2020-08-02"),(7,1302,0,72,0,"2020-08-03"),(7,1356,0,13,45,"2020-08-04"),
        (7,1256,1,28,0,"2020-08-05"),(7,985,0,11,5,"2020-08-06"),(7,934,0,40,0,"2020-08-07"),(7,464,1,65,0,"2020-08-08"),
        (7,930,0,0,0,"2020-08-09"),(7,1385,0,30,0,"2020-08-10"),(7,635,0,38,0,"2020-08-11"),(7,588,2,60,0,"2020-08-12"),
        (7,849,2,40,0,"2020-08-13"),(7,1405,3,10,0,"2020-08-14"),(7,911,2,67,24,"2020-08-15"),(7,1495,0,83,0,"2020-08-16"),
        (7,830,0,47,23,"2020-08-17"),(7,1295,0,52,0,"2020-08-18"),(7,844,1,9,0,"2020-08-19"),(7,1246,0,12,4,"2020-08-20"),
        (7,1044,0,28,0,"2020-08-21"),(7,732,1,0,0,"2020-08-22"),(7,1468,4,13,0,"2020-08-23"),(7,967,0,79,0,"2020-08-24"),
        (7,1953,1,17,19,"2020-08-25"),(7,802,0,24,11,"2020-08-26"),(7,1853,1,71,15,"2020-08-27"),(7,1344,0,61,16,"2020-08-28"),
        (7,1067,2,13,0,"2020-08-29"),(7,1740,0,33,57,"2020-08-30"),(7,1122,0,38,12,"2020-08-31"),
        (8,346,5,75,30,"2020-08-01"),(8,502,6,74,40,"2020-08-02"),(8,482,1,12,62,"2020-08-03"),(8,200,10,63,69,"2020-08-04"),
        (8,386,6,83,29,"2020-08-05"),(8,310,3,62,28,"2020-08-06"),(8,500,3,123,0,"2020-08-07"),(8,284,5,71,31,"2020-08-08"),
        (8,305,12,42,19,"2020-08-09"),(8,375,13,80,15,"2020-08-10"),(8,319,5,84,29,"2020-08-11"),(8,249,4,42,53,"2020-08-12"),
        (8,401,10,26,0,"2020-08-13"),(8,281,15,37,76,"2020-08-14"),(8,318,7,86,34,"2020-08-15"),(8,394,8,39,12,"2020-08-16"),
        (8,250,8,56,63,"2020-08-17"),(8,272,4,42,59,"2020-08-18"),(8,492,1,70,46,"2020-08-19"),(8,365,3,51,33,"2020-08-20"),
        (8,284,2,12,25,"2020-08-21"),(8,266,5,50,15,"2020-08-22"),(8,172,1,67,12,"2020-08-23"),(8,402,0,68,8,"2020-08-24"),
        (8,265,4,103,31,"2020-08-25"),(8,388,9,62,43,"2020-08-26"),(8,193,1,48,43,"2020-08-27"),(8,95,6,77,18,"2020-08-28"),
        (8,192,0,93,22,"2020-08-29"),(8,293,12,71,55,"2020-08-30"),(8,174,1,52,36,"2020-08-31"),
        (9,1304,0,13,0,"2020-08-01"),(9,720,0,11,5,"2020-08-02"),(9,1111,1,8,12,"2020-08-03"),(9,1760,1,5,1,"2020-08-04"),
        (9,1502,1,0,3,"2020-08-05"),(9,1455,2,21,0,"2020-08-06"),(9,850,2,7,0,"2020-08-07"),(9,799,2,7,7,"2020-08-08"),
        (9,623,0,5,0,"2020-08-09"),(9,1294,1,5,0,"2020-08-10"),(9,950,0,4,0,"2020-08-11"),(9,923,0,15,0,"2020-08-12"),
        (9,730,0,8,0,"2020-08-13"),(9,823,0,0,0,"2020-08-14"),(9,957,0,0,7,"2020-08-15"),(9,829,0,0,2,"2020-08-16"),
        (9,945,0,0,5,"2020-08-17"),(9,392,0,0,3,"2020-08-18"),(9,1028,0,3,12,"2020-08-19"),(9,729,0,0,0,"2020-08-20"),
        (9,920,4,4,0,"2020-08-21"),(9,720,1,2,9,"2020-08-22"),(9,1932,0,8,27,"2020-08-23"),(9,999,3,0,6,"2020-08-24"),
        (9,869,0,8,0,"2020-08-25"),(9,1830,0,20,15,"2020-08-26"),(9,1234,1,0,0,"2020-08-27"),(9,1885,4,24,13,"2020-08-28"),
        (9,888,2,13,0,"2020-08-29"),(9,929,1,7,6,"2020-08-30"),(9,606,2,37,0,"2020-08-31"),
        (10,482,3,35,20,"2020-08-01"),(10,418,0,32,18,"2020-08-02"),(10,382,3,22,24,"2020-08-03"),(10,212,2,30,16,"2020-08-04"),
        (10,510,7,43,6,"2020-08-05"),(10,399,1,43,25,"2020-08-06"),(10,593,0,23,25,"2020-08-07"),(10,392,2,4,25,"2020-08-08"),
        (10,692,0,13,17,"2020-08-09"),(10,312,0,2,15,"2020-08-10"),(10,492,0,13,27,"2020-08-11"),(10,433,0,32,18,"2020-08-12"),
        (10,582,2,0,29,"2020-08-13"),(10,422,3,31,6,"2020-08-14"),(10,598,1,4,46,"2020-08-15"),(10,510,0,50,12,"2020-08-16"),
        (10,382,5,34,0,"2020-08-17"),(10,628,0,0,21,"2020-08-18"),(10,428,4,22,37,"2020-08-19"),(10,413,1,12,34,"2020-08-20"),
        (10,318,6,6,7,"2020-08-21"),(10,700,2,16,30,"2020-08-22"),(10,1021,5,11,32,"2020-08-23"),(10,284,8,0,28,"2020-08-24"),
        (10,492,2,3,6,"2020-08-25"),(10,238,0,21,16,"2020-08-26"),(10,453,0,0,17,"2020-08-27"),(10,333,10,6,14,"2020-08-28"),
        (10,609,0,14,16,"2020-08-29"),(10,395,5,0,7,"2020-08-30"),(10,530,1,9,0,"2020-08-31"),
        (11,2134,0,8,11,"2020-08-01"),(11,1093,0,18,12,"2020-08-02"),(11,1467,0,15,2,"2020-08-03"),(11,1760,0,4,9,"2020-08-04"),
        (11,1482,0,12,0,"2020-08-05"),(11,1004,0,0,2,"2020-08-06"),(11,2402,1,0,8,"2020-08-07"),(11,838,1,0,0,"2020-08-08"),
        (11,922,0,0,0,"2020-08-09"),(11,983,2,2,3,"2020-08-10"),(11,1803,1,7,10,"2020-08-11"),(11,927,0,12,4,"2020-08-12"),
        (11,1450,1,16,3,"2020-08-13"),(11,889,0,7,9,"2020-08-14"),(11,1234,0,28,0,"2020-08-15"),(11,1583,0,17,0,"2020-08-16"),
        (11,1503,0,0,0,"2020-08-17"),(11,798,1,0,3,"2020-08-18"),(11,1652,0,0,0,"2020-08-19"),(11,1362,0,0,6,"2020-08-20"),
        (11,1793,0,0,0,"2020-08-21"),(11,1928,0,0,0,"2020-08-22"),(11,821,0,7,11,"2020-08-23"),(11,1278,1,0,0,"2020-08-24"),
        (11,1112,0,7,18,"2020-08-25"),(11,1356,0,19,4,"2020-08-26"),(11,629,2,15,0,"2020-08-27"),(11,1029,2,8,13,"2020-08-28"),
        (11,829,0,4,8,"2020-08-29"),(11,1224,3,0,0,"2020-08-30"),(11,492,0,6,8,"2020-08-31"),
        (12,92,5,32,30,"2020-08-01"),(12,201,1,21,17,"2020-08-02"),(12,218,12,8,13,"2020-08-03"),(12,49,2,21,4,"2020-08-04"),
        (12,134,8,16,40,"2020-08-05"),(12,192,8,19,23,"2020-08-06"),(12,167,15,9,6,"2020-08-07"),(12,92,12,24,16,"2020-08-08"),
        (12,27,1,39,21,"2020-08-09"),(12,78,17,27,15,"2020-08-10"),(12,51,8,17,8,"2020-08-11"),(12,106,14,12,13,"2020-08-12"),
        (12,39,5,27,72,"2020-08-13"),(12,90,4,18,31,"2020-08-14"),(12,73,9,10,0,"2020-08-15"),(12,176,12,9,7,"2020-08-16"),
        (12,182,0,17,52,"2020-08-17"),(12,91,8,12,8,"2020-08-18"),(12,82,7,0,6,"2020-08-19"),(12,234,6,0,15,"2020-08-20"),
        (12,76,9,10,29,"2020-08-21"),(12,108,6,18,8,"2020-08-22"),(12,72,9,14,25,"2020-08-23"),(12,188,2,7,8,"2020-08-24"),
        (12,111,13,11,17,"2020-08-25"),(12,115,0,7,9,"2020-08-26"),(12,63,6,30,4,"2020-08-27"),(12,177,0,17,22,"2020-08-28"),
        (12,124,12,20,28,"2020-08-29"),(12,100,3,0,13,"2020-08-30"),(12,94,14,19,13,"2020-08-31"),  
        (13,530,5,15,12,"2020-08-01"),(13,736,4,0,0,"2020-08-02"),(13,649,4,13,22,"2020-08-03"),(13,698,6,0,0,"2020-08-04"),
        (13,721,2,17,23,"2020-08-05"),(13,816,2,18,6,"2020-08-06"),(13,701,3,19,18,"2020-08-07"),(13,726,7,0,17,"2020-08-08"),
        (13,394,3,9,18,"2020-08-09"),(13,793,0,9,2,"2020-08-10"),(13,612,9,0,5,"2020-08-11"),(13,746,2,14,8,"2020-08-12"),
        (13,455,1,0,14,"2020-08-13"),(13,439,1,21,7,"2020-08-14"),(13,682,8,7,9,"2020-08-15"),(13,893,0,19,23,"2020-08-16"),
        (13,801,0,13,0,"2020-08-17"),(13,399,7,2,15,"2020-08-18"),(13,723,0,16,15,"2020-08-19"),(13,487,0,21,22,"2020-08-20"),
        (13,739,6,20,8,"2020-08-21"),(13,465,6,3,5,"2020-08-22"),(13,788,3,7,13,"2020-08-23"),(13,589,5,12,6,"2020-08-24"),
        (13,666,2,13,3,"2020-08-25"),(13,582,0,0,16,"2020-08-26"),(13,625,7,12,12,"2020-08-27"),(13,501,3,4,8,"2020-08-28"),
        (13,615,0,9,14,"2020-08-29"),(13,611,3,4,16,"2020-08-30"),(13,791,4,9,28,"2020-08-31"),
        (14,983,1,34,61,"2020-08-01"),(14,999,1,78,32,"2020-08-02"),(14,1039,0,34,19,"2020-08-03"),(14,772,1,53,31,"2020-08-04"),
        (14,1023,0,29,27,"2020-08-05"),(14,983,0,60,31,"2020-08-06"),(14,1324,0,28,29,"2020-08-07"),(14,861,3,49,30,"2020-08-08"),
        (14,1034,0,38,42,"2020-08-09"),(14,836,0,0,46,"2020-08-10"),(14,927,0,35,44,"2020-08-11"),(14,697,2,26,8,"2020-08-12"),
        (14,802,0,55,12,"2020-08-13"),(14,782,0,13,34,"2020-08-14"),(14,915,1,50,18,"2020-08-15"),(14,810,0,20,17,"2020-08-16"),
        (14,709,2,71,22,"2020-08-17"),(14,888,2,12,19,"2020-08-18"),(14,827,0,38,33,"2020-08-19"),(14,1293,0,9,29,"2020-08-20"),
        (14,1823,0,59,0,"2020-08-21"),(14,1083,0,23,12,"2020-08-22"),(14,1381,1,29,19,"2020-08-23"),(14,1173,1,13,39,"2020-08-24"),
        (14,1432,0,24,47,"2020-08-25"),(14,1063,0,42,19,"2020-08-26"),(14,1402,0,33,59,"2020-08-27"),(14,929,0,13,74,"2020-08-28"),
        (14,1304,0,14,59,"2020-08-29"),(14,1145,3,33,30,"2020-08-30"),(14,1102,0,72,0,"2020-08-31"),
        (15,539,3,13,26,"2020-08-01"),(15,617,0,27,14,"2020-08-02"),(15,729,0,18,10,"2020-08-03"),(15,521,1,0,24,"2020-08-04"),
        (15,491,0,14,22,"2020-08-05"),(15,719,4,7,27,"2020-08-06"),(15,821,0,8,11,"2020-08-07"),(15,594,1,32,7,"2020-08-08"),
        (15,928,4,8,16,"2020-08-09"),(15,830,3,0,16,"2020-08-10"),(15,902,3,0,6,"2020-08-11"),(15,492,4,22,12,"2020-08-12"),
        (15,619,6,9,14,"2020-08-13"),(15,715,2,25,2,"2020-08-14"),(15,762,5,12,1,"2020-08-15"),(15,378,7,3,24,"2020-08-16"),
        (15,729,8,17,0,"2020-08-17"),(15,785,6,4,8,"2020-08-18"),(15,681,12,17,27,"2020-08-19"),(15,402,5,8,2,"2020-08-20"),
        (15,800,2,0,6,"2020-08-21"),(15,516,8,0,0,"2020-08-22"),(15,535,3,9,5,"2020-08-23"),(15,501,4,7,1,"2020-08-24"),
        (15,910,1,2,6,"2020-08-25"),(15,684,9,21,9,"2020-08-26"),(15,587,2,18,12,"2020-08-27"),(15,610,2,14,18,"2020-08-28"),
        (15,527,5,16,0,"2020-08-29"),(15,901,2,13,15,"2020-08-30"),(15,534,0,6,14,"2020-08-31"),
        (17,1648,0,3,0,"2020-08-01"),(17,1849,0,6,11,"2020-08-02"),(17,1453,0,12,10,"2020-08-03"),(17,921,0,13,5,"2020-08-04"),
        (17,1623,0,0,0,"2020-08-05"),(17,1632,0,0,0,"2020-08-06"),(17,1829,0,2,9,"2020-08-07"),(17,1834,1,0,5,"2020-08-08"),
        (17,2341,0,0,7,"2020-08-09"),(17,2483,2,0,8,"2020-08-10"),(17,1712,1,2,3,"2020-08-11"),(17,1749,0,12,9,"2020-08-12"),
        (17,1232,0,0,0,"2020-08-13"),(17,2013,0,2,0,"2020-08-14"),(17,1144,1,4,7,"2020-08-15"),(17,1334,2,21,0,"2020-08-16"),
        (17,1552,0,0,0,"2020-08-17"),(17,1934,0,3,5,"2020-08-18"),(17,1536,0,11,2,"2020-08-19"),(17,1346,0,3,1,"2020-08-20"),
        (17,920,1,8,6,"2020-08-21"),(17,1235,1,10,2,"2020-08-22"),(17,2184,1,13,8,"2020-08-23"),(17,1765,3,6,8,"2020-08-24"),
        (17,1928,0,5,4,"2020-08-25"),(17,1321,0,5,1,"2020-08-26"),(17,2562,0,0,18,"2020-08-27"),(17,1113,0,11,11,"2020-08-28"),
        (17,1057,0,3,0,"2020-08-29"),(17,1022,0,0,7,"2020-08-30"),(17,1240,0,0,0,"2020-08-31"),
        (18,782,0,46,54,"2020-08-01"),(18,492,6,40,31,"2020-08-02"),(18,362,1,49,28,"2020-08-03"),(18,593,7,12,34,"2020-08-04"),
        (18,892,2,24,62,"2020-08-05"),(18,584,3,4,52,"2020-08-06"),(18,392,0,21,35,"2020-08-07"),(18,793,1,29,29,"2020-08-08"),
        (18,1639,4,32,23,"2020-08-09"),(18,493,0,62,14,"2020-08-10"),(18,418,4,44,49,"2020-08-11"),(18,632,0,85,24,"2020-08-12"),
        (18,622,1,18,27,"2020-08-13"),(18,302,4,21,19,"2020-08-14"),(18,472,2,20,38,"2020-08-15"),(18,535,5,37,19,"2020-08-16"),
        (18,522,5,9,49,"2020-08-17"),(18,592,7,42,35,"2020-08-18"),(18,520,3,19,37,"2020-08-19"),(18,589,5,20,29,"2020-08-20"),
        (18,482,6,19,18,"2020-08-21"),(18,857,4,19,72,"2020-08-22"),(18,539,8,4,13,"2020-08-23"),(18,470,1,44,31,"2020-08-24"),
        (18,527,2,49,38,"2020-08-25"),(18,496,0,23,32,"2020-08-26"),(18,611,6,39,19,"2020-08-27"),(18,593,2,54,40,"2020-08-28"),
        (18,812,0,75,21,"2020-08-29"),(18,523,2,29,42,"2020-08-30"),(18,389,0,82,14,"2020-08-31"),
        (19,1345,1,18,21,"2020-08-01"),(19,1520,3,20,18,"2020-08-02"),(19,1037,4,19,12,"2020-08-03"),(19,1628,0,18,11,"2020-08-04"),
        (19,1129,0,14,7,"2020-08-05"),(19,1045,0,19,19,"2020-08-06"),(19,1122,3,14,15,"2020-08-07"),(19,1072,1,23,14,"2020-08-08"),
        (19,1249,2,19,17,"2020-08-09"),(19,1294,4,14,16,"2020-08-10"),(19,972,0,20,13,"2020-08-11"),(19,882,0,19,8,"2020-08-12"),
        (19,938,2,20,8,"2020-08-13"),(19,1183,0,9,8,"2020-08-14"),(19,927,2,28,19,"2020-08-15"),(19,926,1,10,7,"2020-08-16"),
        (19,910,0,11,16,"2020-08-17"),(19,1009,3,15,20,"2020-08-18"),(19,899,0,7,9,"2020-08-19"),(19,935,4,12,19,"2020-08-20"),
        (19,888,0,12,22,"2020-08-21"),(19,1212,2,11,18,"2020-08-22"),(19,1322,3,11,15,"2020-08-23"),(19,1088,2,6,13,"2020-08-24"),
        (19,1420,4,21,29,"2020-08-25"),(19,988,0,17,9,"2020-08-26"),(19,1129,0,8,10,"2020-08-27"),(19,1333,0,17,16,"2020-08-28"),
        (19,1333,1,27,19,"2020-08-29"),(19,892,1,10,4,"2020-08-30"),(19,1341,2,13,9,"2020-08-31"),
        (21,1483,0,53,34,"2020-08-01"),(21,1938,0,34,42,"2020-08-02"),(21,1305,0,78,22,"2020-08-03"),(21,1324,0,25,39,"2020-08-04"),
        (21,1837,0,32,28,"2020-08-05"),(21,2183,0,21,38,"2020-08-06"),(21,1735,1,26,43,"2020-08-07"),(21,1253,3,41,19,"2020-08-08"),
        (21,983,1,35,20,"2020-08-09"),(21,1634,3,17,29,"2020-08-10"),(21,1538,0,33,19,"2020-08-11"),(21,1653,0,55,67,"2020-08-12"),
        (21,1038,0,21,17,"2020-08-13"),(21,826,0,20,17,"2020-08-14"),(21,1492,1,82,34,"2020-08-15"),(21,1403,0,31,56,"2020-08-16"),
        (21,1392,0,27,19,"2020-08-17"),(21,1542,0,17,34,"2020-08-18"),(21,937,0,77,40,"2020-08-19"),(21,1839,0,20,42,"2020-08-20"),
        (21,1837,0,31,30,"2020-08-21"),(21,1639,0,32,46,"2020-08-22"),(21,827,1,23,37,"2020-08-23"),(21,1432,1,45,39,"2020-08-24"),
        (21,803,0,14,52,"2020-08-25"),(21,1329,0,82,52,"2020-08-26"),(21,1748,0,81,12,"2020-08-27"),(21,1522,1,50,50,"2020-08-28"),
        (21,1432,2,39,42,"2020-08-29"),(21,1238,1,68,51,"2020-08-30"),(21,2242,0,54,32,"2020-08-31"),
        (22,2403,0,52,43,"2020-08-01"),(22,2912,0,61,30,"2020-08-02"),(22,1098,1,83,21,"2020-08-03"),(22,2183,2,61,39,"2020-08-04"),
        (22,2421,0,19,37,"2020-08-05"),(22,2182,1,18,29,"2020-08-06"),(22,1128,0,66,55,"2020-08-07"),(22,2382,0,53,48,"2020-08-08"),
        (22,2018,0,22,39,"2020-08-09"),(22,1592,0,28,27,"2020-08-10"),(22,829,0,47,30,"2020-08-11"),(22,2110,0,20,20,"2020-08-12"),
        (22,1732,0,49,28,"2020-08-13"),(22,1037,0,68,21,"2020-08-14"),(22,1638,0,29,17,"2020-08-15"),(22,1044,0,16,44,"2020-08-16"),
        (22,1731,1,8,22,"2020-08-17"),(22,1111,0,72,19,"2020-08-18"),(22,1543,0,35,39,"2020-08-19"),(22,937,0,59,21,"2020-08-20"),
        (22,1372,0,3,53,"2020-08-21"),(22,1762,0,18,35,"2020-08-22"),(22,1472,0,13,28,"2020-08-23"),(22,2493,0,20,46,"2020-08-24"),
        (22,1138,0,13,42,"2020-08-25"),(22,1662,0,9,13,"2020-08-26"),(22,1600,0,95,8,"2020-08-27"),(22,891,0,39,53,"2020-08-28"),
        (22,1240,0,72,31,"2020-08-29"),(22,1144,0,23,38,"2020-08-30"),(22,1593,1,52,29,"2020-08-31"),
        (23,222,2,50,55,"2020-08-01"),(23,62,0,60,32,"2020-08-02"),(23,26,0,30,22,"2020-08-03"),(23,29,1,7,67,"2020-08-04"),
        (23,169,1,62,42,"2020-08-05"),(23,173,0,47,51,"2020-08-06"),(23,135,0,18,48,"2020-08-07"),(23,99,0,50,15,"2020-08-08"),
        (23,82,0,38,46,"2020-08-09"),(23,200,0,23,30,"2020-08-10"),(23,147,0,21,49,"2020-08-11"),(23,178,0,42,21,"2020-08-12"),
        (23,183,0,30,51,"2020-08-13"),(23,144,0,28,36,"2020-08-14"),(23,168,0,30,30,"2020-08-15"),(23,299,2,30,8,"2020-08-16"),
        (23,283,0,41,29,"2020-08-17"),(23,152,2,19,45,"2020-08-18"),(23,291,0,34,31,"2020-08-19"),(23,219,1,18,19,"2020-08-20"),
        (23,212,0,39,44,"2020-08-21"),(23,169,0,29,39,"2020-08-22"),(23,53,0,29,34,"2020-08-23"),(23,178,0,40,49,"2020-08-24"),
        (23,192,1,50,28,"2020-08-25"),(23,98,0,25,45,"2020-08-26"),(23,22,1,32,85,"2020-08-27"),(23,155,0,33,51,"2020-08-28"),
        (23,73,0,40,46,"2020-08-29"),(23,54,1,14,25,"2020-08-30"),(23,35,3,12,90,"2020-08-31"),
        (25,1839,4,42,19,"2020-08-01"),(25,1555,0,60,6,"2020-08-02"),(25,1412,0,21,7,"2020-08-03"),(25,1242,5,29,14,"2020-08-04"),
        (25,1537,2,32,17,"2020-08-05"),(25,1492,1,32,0,"2020-08-06"),(25,1500,1,27,29,"2020-08-07"),(25,1039,2,37,28,"2020-08-08"),
        (25,1392,0,59,8,"2020-08-09"),(25,1637,0,43,12,"2020-08-10"),(25,902,1,22,49,"2020-08-11"),(25,1183,0,81,8,"2020-08-12"),
        (25,892,3,24,13,"2020-08-13"),(25,1228,3,50,32,"2020-08-14"),(25,1182,4,38,7,"2020-08-15"),(25,1340,0,70,21,"2020-08-16"),
        (25,989,0,9,17,"2020-08-17"),(25,1447,0,72,12,"2020-08-18"),(25,1632,0,83,11,"2020-08-19"),(25,1483,3,63,18,"2020-08-20"),
        (25,1892,4,47,20,"2020-08-21"),(25,963,0,52,27,"2020-08-22"),(25,1701,0,28,17,"2020-08-23"),(25,1204,4,57,19,"2020-08-24"),
        (25,792,1,56,19,"2020-08-25"),(25,1284,0,43,12,"2020-08-26"),(25,1132,5,32,19,"2020-08-27"),(25,1874,1,48,2,"2020-08-28"),
        (25,1638,0,72,11,"2020-08-29"),(25,1195,4,50,22,"2020-08-30"),(25,1233,2,58,4,"2020-08-31"),
        (26,412,5,59,42,"2020-08-01"),(26,298,5,77,45,"2020-08-02"),(26,429,7,34,47,"2020-08-03"),(26,404,8,44,60,"2020-08-04"),
        (26,383,13,38,30,"2020-08-05"),(26,288,7,61,15,"2020-08-06"),(26,502,4,17,50,"2020-08-07"),(26,593,6,29,36,"2020-08-08"),
        (26,492,9,28,24,"2020-08-09"),(26,232,13,21,41,"2020-08-10"),(26,489,3,20,29,"2020-08-11"),(26,605,4,39,52,"2020-08-12"),
        (26,502,2,35,41,"2020-08-13"),(26,399,15,23,33,"2020-08-14"),(26,400,3,40,33,"2020-08-15"),(26,472,13,35,49,"2020-08-16"),
        (26,664,8,18,33,"2020-08-17"),(26,477,6,41,38,"2020-08-18"),(26,302,7,67,28,"2020-08-19"),(26,533,2,58,30,"2020-08-20"),
        (26,544,11,55,25,"2020-08-21"),(26,425,3,57,29,"2020-08-22"),(26,239,14,50,33,"2020-08-23"),(26,421,6,62,43,"2020-08-24"),
        (26,372,13,39,59,"2020-08-25"),(26,362,1,14,40,"2020-08-26"),(26,277,3,60,48,"2020-08-27"),(26,388,10,50,50,"2020-08-28"),
        (26,301,8,78,30,"2020-08-29"),(26,311,9,92,3,"2020-08-30"),(26,399,2,67,21,"2020-08-31"),
        (27,183,5,15,37,"2020-08-01"),(27,283,6,3,29,"2020-08-02"),(27,156,9,33,25,"2020-08-03"),(27,253,14,8,39,"2020-08-04"),
        (27,200,7,22,28,"2020-08-05"),(27,109,8,16,21,"2020-08-06"),(27,161,7,18,48,"2020-08-07"),(27,200,5,12,62,"2020-08-08"),
        (27,126,4,13,31,"2020-08-09"),(27,193,0,7,36,"2020-08-10"),(27,98,5,16,53,"2020-08-11"),(27,121,3,17,30,"2020-08-12"),
        (27,255,9,28,20,"2020-08-13"),(27,244,9,37,24,"2020-08-14"),(27,188,11,21,37,"2020-08-15"),(27,249,6,24,24,"2020-08-16"),
        (27,242,1,31,23,"2020-08-17"),(27,237,15,15,57,"2020-08-18"),(27,264,6,7,54,"2020-08-19"),(27,255,1,41,19,"2020-08-20"),
        (27,251,13,7,44,"2020-08-21"),(27,281,2,30,34,"2020-08-22"),(27,338,3,23,43,"2020-08-23"),(27,201,13,8,57,"2020-08-24"),
        (27,277,11,13,38,"2020-08-25"),(27,299,12,28,19,"2020-08-26"),(27,139,1,22,47,"2020-08-27"),(27,267,10,16,40,"2020-08-28"),
        (27,178,8,19,27,"2020-08-29"),(27,177,17,9,45,"2020-08-30"),(27,87,12,11,18,"2020-08-31"),
        (28,1124,14,5,12,"2020-08-01"),(28,1044,7,7,7,"2020-08-02"),(28,755,13,6,6,"2020-08-03"),(28,1533,18,7,2,"2020-08-04"),
        (28,679,18,8,4,"2020-08-05"),(28,677,11,1,8,"2020-08-06"),(28,632,16,7,0,"2020-08-07"),(28,1034,7,3,3,"2020-08-08"),
        (28,575,2,14,4,"2020-08-09"),(28,732,15,25,0,"2020-08-10"),(28,687,2,3,15,"2020-08-11"),(28,543,9,11,16,"2020-08-12"),
        (28,533,7,4,9,"2020-08-13"),(28,789,4,19,17,"2020-08-14"),(28,555,1,14,9,"2020-08-15"),(28,566,12,13,27,"2020-08-16"),
        (28,423,14,3,5,"2020-08-17"),(28,643,9,7,2,"2020-08-18"),(28,502,7,5,5,"2020-08-19"),(28,624,13,1,17,"2020-08-20"),
        (28,1463,5,0,11,"2020-08-21"),(28,611,12,8,11,"2020-08-22"),(28,487,14,3,16,"2020-08-23"),(28,620,8,22,0,"2020-08-24"),
        (28,699,11,14,8,"2020-08-25"),(28,586,10,17,0,"2020-08-26"),(28,679,9,0,4,"2020-08-27"),(28,1111,5,9,6,"2020-08-28"),
        (28,581,8,21,10,"2020-08-29"),(28,581,6,4,13,"2020-08-30"),(28,700,6,7,13,"2020-08-31"),
        (29,1524,0,56,25,"2020-08-01"),(29,987,0,42,39,"2020-08-02"),(29,1439,1,34,38,"2020-08-03"),(29,1188,0,41,37,"2020-08-04"),
        (29,1730,0,48,39,"2020-08-05"),(29,1771,0,49,43,"2020-08-06"),(29,1378,0,41,29,"2020-08-07"),(29,1263,0,33,59,"2020-08-08"),
        (29,1930,1,27,57,"2020-08-09"),(29,1849,1,38,36,"2020-08-10"),(29,1598,0,39,37,"2020-08-11"),(29,1730,0,21,68,"2020-08-12"),
        (29,1847,0,64,19,"2020-08-13"),(29,1667,0,30,68,"2020-08-14"),(29,1702,0,68,31,"2020-08-15"),(29,2740,0,28,46,"2020-08-16"),
        (29,1222,0,30,34,"2020-08-17"),(29,1493,4,27,59,"2020-08-18"),(29,1832,2,84,18,"2020-08-19"),(29,2489,0,34,32,"2020-08-20"),
        (29,1352,2,74,40,"2020-08-21"),(29,1339,0,18,22,"2020-08-22"),(29,1440,0,36,36,"2020-08-23"),(29,2009,2,50,56,"2020-08-24"),
        (29,1667,0,55,36,"2020-08-25"),(29,1028,0,37,65,"2020-08-26"),(29,927,0,25,39,"2020-08-27"),(29,2021,0,33,13,"2020-08-28"),
        (29,1599,0,61,38,"2020-08-29"),(29,1088,0,70,39,"2020-08-30"),(29,1073,1,47,27,"2020-08-31"),
        (30,329,2,20,45,"2020-08-01"),(30,222,8,10,47,"2020-08-02"),(30,173,12,4,41,"2020-08-03"),(30,177,13,7,52,"2020-08-04"),
        (30,710,6,10,42,"2020-08-05"),(30,334,2,2,37,"2020-08-06"),(30,123,16,8,39,"2020-08-07"),(30,189,15,8,46,"2020-08-08"),
        (30,400,5,7,29,"2020-08-09"),(30,354,6,20,78,"2020-08-10"),(30,394,9,6,79,"2020-08-11"),(30,207,9,11,83,"2020-08-12"),
        (30,329,0,4,47,"2020-08-13"),(30,378,6,4,44,"2020-08-14"),(30,338,14,7,39,"2020-08-15"),(30,284,2,6,93,"2020-08-16"),
        (30,284,0,6,36,"2020-08-17"),(30,312,13,1,39,"2020-08-18"),(30,267,17,6,29,"2020-08-19"),(30,351,8,15,43,"2020-08-20"),
        (30,244,4,0,67,"2020-08-21"),(30,200,4,14,29,"2020-08-22"),(30,313,11,12,58,"2020-08-23"),(30,273,14,14,55,"2020-08-24"),
        (30,285,12,11,39,"2020-08-25"),(30,173,5,9,55,"2020-08-26"),(30,461,10,13,64,"2020-08-27"),(30,290,17,29,61,"2020-08-28"),
        (30,197,7,8,51,"2020-08-29"),(30,183,15,4,64,"2020-08-30"),(30,245,16,50,43,"2020-08-31")]

df_consumos_diarios = spark.createDataFrame(data, columns)
#df_consumos_diarios.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_consumos_diarios')
df_consumos_diarios.show()



# Para escribir una tabla de databricks no es necesario especificar la base de datos porque se guardan en la BBDD por defecto (default).
# Trabajando desde el ecosistema Cloudera, es necesario especificar: saveAsTable("database_name.a_consumos")
#df_consumos_diarios.write.mode("overwrite").saveAsTable('a_consumos')


# Escribir dataframe como CSV en el filesystem de Databricks
#df_consumos_diarios.write.format('com.databricks.spark.csv').save('dbfs:/FileStore/imf_big_data',header='true')

+----------+----------------+------------+----------------------+---------------------+----------+
|id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|     fecha|
+----------+----------------+------------+----------------------+---------------------+----------+
|         1|             664|           3|                    25|                    0|2020-08-01|
|         1|             943|           0|                    12|                    5|2020-08-02|
|         1|            1035|           1|                    46|                   10|2020-08-03|
|         1|             760|           3|                    17|                    0|2020-08-04|
|         1|            1409|           1|                    31|                    4|2020-08-05|
|         1|            1627|           0|                    20|                    0|2020-08-06|
|         1|             739|           3|                    21|                    0|2020-08-07|
|         

###Comandos principales de PySpark###
Creación de columnas y agregaciones que irán enriqueciendo nuestros datos y obteniendo diferentes insights de nuestras tablas de entrada

**1. Mostrar los metadatos (tipos de datos) de cada columna del DataFrame facturas_mes_ant** (printSchema)

In [15]:
# Importante especificar el doble paréntesis después del método, lo cual indica que este método de la clase DataFrame no recibe ningún parámetro
df_facturas_mes_ant.printSchema()

root
 |-- id_cliente: long (nullable = true)
 |-- id_oferta: long (nullable = true)
 |-- importe: double (nullable = true)
 |-- fecha: string (nullable = true)



**2. Mostrar por pantalla el número TOTAL de contratos existentes entre los clientes de la muestra el mes anterior (julio 2020)** (count)

In [ ]:
# En primer lugar, debemos tener en cuenta que un mismo clientes puede tener varios contratos.
# Como se nos dice el total de contratos del mes_anterior, simplemente será necesario contar el número de registros del DataFrame facturas_mes_ant
df_facturas_mes_ant.count()
print("El número TOTAL de contratos de la muestra existentes el mes anterior era de: " + str(df_facturas_mes_ant.count()))
# Nota: Importante tener en cuenta que para mostrar el resultado del count() en un comando print es necesario castear a string el resultado del comando, que es un entero (Int)

El número TOTAL de contratos de la muestra existentes el mes anterior era de: 34


**3. Mostrar por pantalla el número de clientes de la muestra que ya existían en el mes anterior** (dropDuplicates)

In [18]:
#df_facturas_mes_ant.dropDuplicates(["id_cliente"]).count()
print("El número de clientes de la muestra que existían el mes anterior es de: " + str(df_facturas_mes_ant.dropDuplicates(["id_cliente"]).count()))

El número de clientes de la muestra que existían el mes anterior es de: 25


**4. Crear una nueva variable "grupo_edad" que agrupe a los clientes, tanto del mes_actual como del mes_anterior en 4 rangos según su edad asignando valores del 1 - 4 segun el rango al que pertenezcan (18-25 (1), 26-40 (2), 41-65 (3), >65 (4))** (left join, withColumn, when/otherwise, lit)

In [32]:
# En primer lugar, tenemos que tener en cuenta que hay que crear la nueva variable "grupo_edad" tanto para el dataframe del mes anterior (df_facturas_mes_ant) como en el del mes actual (df_facturas_mes_actual), puesto que aplica a ambos.
# Sin embargo, el campo "edad" no existe en estos dataframes, sino que viene en el dataframe que contiene información de los clientes (df_clientes). Por tanto, necesitamos agregar a las dos tablas esta información mediante el uso del comando left_join.

# Renombramos los dataframes de clientes y facturas para que sigamos manteniendo los dataframes originales y usarlos posteriormente.

# 1. Seleccionamos del df de clientes solo las columnas necesarias: id_cliente (columna por la que se cruzan ambas tablas) y edad.
df_clientes_join = df_clientes.select("id_cliente","edad")

In [33]:
# 2. Realizamos el join. Es de tipo "left" porque queremos agregar información a los dataframes de facturas, pero mantener todos los registros (filas) y columnas del mismo dataframe.
df_new_mes_ant = df_facturas_mes_ant.join(df_clientes_join,["id_cliente"],"left").orderBy("id_cliente")
df_new_mes_act = df_facturas_mes_actual.join(df_clientes_join,["id_cliente"],"left").orderBy("id_cliente")

df_new_mes_ant.show()
df_new_mes_act.show()

+----------+---------+-------+----------+----+
|id_cliente|id_oferta|importe|     fecha|edad|
+----------+---------+-------+----------+----+
|         1|        9|   36.5|2020-07-31|  26|
|         1|        6|  118.6|2020-07-31|  26|
|         1|        6|  124.5|2020-07-31|  26|
|         2|        4|   95.0|2020-07-31|  58|
|         3|        5|  102.5|2020-07-31|  68|
|         5|        9|   40.5|2020-07-31|  41|
|         5|        7|   80.5|2020-07-31|  41|
|         6|        8|   84.0|2020-07-31|  74|
|         7|        4|   95.0|2020-07-31|  19|
|         8|        5|  102.5|2020-07-31|  49|
|        10|        6|  124.5|2020-07-31|  40|
|        11|        3|   80.0|2020-07-31|  20|
|        12|        1|   70.0|2020-07-31|  75|
|        13|        4|   95.0|2020-07-31|  53|
|        14|        9|   42.5|2020-07-31|  30|
|        15|        2|   78.5|2020-07-31|  52|
|        15|        6|  115.0|2020-07-31|  52|
|        16|        1|   70.0|2020-07-31|  49|
|        17| 

In [34]:
# Habrá ocasiones en las que, en vez de agregar información de otra tabla como en este caso, lo que queremos es aplicar un tratamiento solo a una parte de los registros, por ejemplo a aquellos clientes que ya existían en el mes anterior y que siguen existiendo en este mes, pero al mismo tiempo queremos mantener el resto de registros.

# Para ello es necesario hacer un left join también, pero incluimos una columna auxiliar en el DF del mes anterior para saber que registros han hecho match entre las dos tablas (los que no hayan hecho match tendrán un valor "null" en la columna auxiliar). 
# Con esta columna, luego será posible aplicar el tratamiento sólo a los clientes deseados.

#df_facturas_mes_actual_flg = df_facturas_mes_actual.join(df_facturas_mes_ant.select("id_cliente").withColumn("flg_mes_ant", lit(1)),["id_cliente"],"left") \
#.dropDuplicates(["id_cliente","id_oferta","importe"]).orderBy("id_cliente")

#df_facturas_mes_actual_flg.show()                                   

IndentationError: unexpected indent (585408968.py, line 7)

In [35]:
# 3. Procedemos a crear la nueva columna "grupo_edad"
# En primer lugar, es necesario importar todas las funciones de pyspark sql, puesto que si no al ejecutar nos dará un error porque Spark no es capaz de reconocer los comandos when,col,lit...
from pyspark.sql.functions import *
# En PySpark, cuando aplicamos el comando withColumn y lo asignamos a la misma variable, lo que está ocurriendo NO es que el dataframe inicial es mutable, sino que se está creando un nuevo objeto DF y hacemos que la referencia de la variable cambie y pase a apuntar a este nuevo DF creado.
df_new_facturas_mes_ant = df_new_mes_ant.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))

df_new_facturas_mes_actual = df_new_mes_act.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))

df_new_facturas_mes_ant.show()
df_new_facturas_mes_actual.show()
# En este caso como la columna "grupo_edad" es nueva, en el otherwise tenemos que establecer un valor de tipo Column, para lo cual es necesario poner lit(). Sin embargo, generalmente en un proceso ETL con mucha lógica de negocio, suele ocurrir que si una columna del DF cambia de valor, sea necesario modificar el valor de otra columna. En este caso, el otherwise para esta última variable a modificar sería del estilo: ""  

+----------+---------+-------+----------+----+----------+
|id_cliente|id_oferta|importe|     fecha|edad|grupo_edad|
+----------+---------+-------+----------+----+----------+
|         1|        9|   36.5|2020-07-31|  26|         2|
|         1|        6|  118.6|2020-07-31|  26|         2|
|         1|        6|  124.5|2020-07-31|  26|         2|
|         2|        4|   95.0|2020-07-31|  58|         3|
|         3|        5|  102.5|2020-07-31|  68|         4|
|         5|        9|   40.5|2020-07-31|  41|         3|
|         5|        7|   80.5|2020-07-31|  41|         3|
|         6|        8|   84.0|2020-07-31|  74|         4|
|         7|        4|   95.0|2020-07-31|  19|         1|
|         8|        5|  102.5|2020-07-31|  49|         3|
|        10|        6|  124.5|2020-07-31|  40|         2|
|        11|        3|   80.0|2020-07-31|  20|         1|
|        12|        1|   70.0|2020-07-31|  75|         4|
|        13|        4|   95.0|2020-07-31|  53|         3|
|        14|  

**5. Obtener un dataframe que contenga los clientes que existían el mes anterior y se han dado de baja este mes, con su nombre, edad, sexo, provincia y país** (select, leftanti join)

In [40]:
# clientes que ya no están en la compañía pero sí estuvieron el mes anterior. Es decir, están en A pero no en B.
#df_bajas=df_facturas_mes_ant.join(df_facturas_mes_actual,["id_cliente"],"left_anti").orderBy("id_cliente")
#df_bajas.show()

In [41]:
#dataframe = df_bajas.join(df_clientes,["id_cliente"],"left").orderBy("id_cliente")
#dataframe.show()

In [42]:
# Tenemos que cruzar el DF de clientes con el facturas del mes anterior para agregar la informacion de nombre, sexo y edad a cada id_cliente, y luego ejecutar un join de tipo leftanti para quedarnos con los clientes que estaban en el mes anterior y se han dado de baja en el actual
df_facturas_mes_ant_clientes = df_facturas_mes_ant.select("id_cliente").join(df_clientes,["id_cliente"],"left")
df_clientes_baja = df_facturas_mes_ant_clientes.join(df_facturas_mes_actual.select("id_cliente"),["id_cliente"],"leftanti")
df_clientes_baja.show()

+----------+-------------+----+----+----------+------+
|id_cliente|       nombre|edad|sexo| provincia|  pais|
+----------+-------------+----+----+----------+------+
|        16|Cristina Sanz|  49|   F|    Madrid|España|
|        20|   Oscar Rico|  22|   M|Valladolid|España|
|        24| Ruben Garcia|  68|   M|   Sevilla|España|
+----------+-------------+----+----+----------+------+



**6. Identificar los clientes que continúan en la compañía este mes y ya estaban en el mes anterior** (inner join)

In [43]:
# Ahora tenemos que encontrar los clientes que concurran en el mes anterior y el actual, para lo cual se utiliza un inner join.
df_clientes_fidelizacion = df_facturas_mes_ant.select("id_cliente").join(df_clientes,["id_cliente"],"left").join(df_facturas_mes_actual,["id_Cliente"],"inner")
df_clientes_fidelizacion.show()
# Nota: es importante que al menos uno de los DF del join tenga un registro único por clave de agrupación, ya que si no obtendremos una explosión de registros:

+----------+----------------+----+----+-----------+--------+---------+-------+----------+
|id_cliente|          nombre|edad|sexo|  provincia|    pais|id_oferta|importe|     fecha|
+----------+----------------+----+----+-----------+--------+---------+-------+----------+
|         6|    David Roldan|  74|   M|Guadalajara|  Mexico|        8|   84.0|2020-08-31|
|         5|      Ana Robles|  41|   F| Valladolid|  España|        7|   80.5|2020-08-31|
|         5|      Ana Robles|  41|   F| Valladolid|  España|        7|   80.5|2020-08-31|
|         5|      Ana Robles|  41|   F| Valladolid|  España|        9|   40.5|2020-08-31|
|         5|      Ana Robles|  41|   F| Valladolid|  España|        9|   40.5|2020-08-31|
|         1|     Pablo Perez|  26|   M|     Madrid|  España|        6|  124.5|2020-08-31|
|         1|     Pablo Perez|  26|   M|     Madrid|  España|        6|  124.5|2020-08-31|
|         1|     Pablo Perez|  26|   M|     Madrid|  España|        6|  124.5|2020-08-31|
|         

**7. Obtener un nuevo DF que muestre cuántos clientes del mes anterior hay en cada uno de los 4 grupos de la columna "grupo_edad"** (groupBy, aggregate, count, alias)

In [46]:
# Eliminamos duplicados para quedarnos en el DF de facturas del mes anterior con un unico registro por cada cliente.
# Agrupamos por la columna "grupo_edad" y contamos cuántos registros hay en cada grupo
df_facturas_mes_ant_group = df1.dropDuplicates(["id_cliente"]).groupBy("grupo_edad").agg(count("grupo_edad").alias("clientes_grupo_edad")).orderBy("grupo_edad").show()
# Observamos que, si sumamos todos los valores de la columna "clientes_grupo_edad" obtenemos un total de 26 clientes que existían ya en el mes anterior. 

+----------+-------------------+
|grupo_edad|clientes_grupo_edad|
+----------+-------------------+
|         1|                  5|
|         2|                  5|
|         3|                  9|
|         4|                  6|
+----------+-------------------+



**8. Crear un nuevo DF que muestre los clientes, con nombre y apellido, que tuvieran más de un contrato en el mes anterior con la compañía, mostrando además el número de contratos que tenían.** (left join, row_number, window rowsBetween)

Forma sencilla.

In [69]:
#df_facturas_mes_ant.show()
df_contador = df_facturas_mes_ant.groupBy("id_cliente").count().withColumnRenamed("count","número_contratos")#contador de frecuencias
df_mas_de_un_contrato = df_contador.filter(col("count")>1).na.drop()

select = df_clientes.select("nombre","id_cliente")
df_mas_de_un_contrato.join(select,["id_cliente"],"left").orderBy("id_cliente").show()

+----------+----------------+-----------------+
|id_cliente|número_contratos|           nombre|
+----------+----------------+-----------------+
|         1|               3|      Pablo Perez|
|         5|               2|       Ana Robles|
|        15|               2|Cristian Cuadrado|
|        18|               2|       Laura Luiz|
|        22|               2|     Clara Suarez|
|        25|               2|     Ines Barcero|
|        27|               2|   Roberta Varado|
|        30|               2|    Marta Rodrigo|
+----------+----------------+-----------------+



Forma más complicada.

In [76]:
# Por un lado, la información correspondiente al nombre del cliente la tenemos en el dataframe df_clientes
# Por otro, tengo que averiguar qué clientes tenían más de un contrato con la compañía el mes anterior a partir 

# EN primer lugar, agregamos la información de ambas tablas como hemos hecho anteriormente (nombre).
df_clientes_nombres = df_clientes.select("id_cliente","nombre") 
df_facturas_mes_ant = df_facturas_mes_ant.join(df_clientes_nombres,["id_cliente"],"left").orderBy("id_cliente")

df_facturas_mes_ant.show()

+----------+---------+-------+----------+-----------------+---------------+---------------+-----------------+
|id_cliente|id_oferta|importe|     fecha|           nombre|count_contratos|total_contratos|           nombre|
+----------+---------+-------+----------+-----------------+---------------+---------------+-----------------+
|         1|        9|   36.5|2020-07-31|      Pablo Perez|              3|              3|      Pablo Perez|
|         1|        6|  118.6|2020-07-31|      Pablo Perez|              2|              3|      Pablo Perez|
|         1|        6|  124.5|2020-07-31|      Pablo Perez|              1|              3|      Pablo Perez|
|         2|        4|   95.0|2020-07-31|  Eduardo Redondo|              1|              1|  Eduardo Redondo|
|         3|        5|  102.5|2020-07-31|  Roberto Salazar|              1|              1|  Roberto Salazar|
|         5|        9|   40.5|2020-07-31|       Ana Robles|              2|              2|       Ana Robles|
|         

In [73]:
# Obtenemos una columna auxiliar que nos permita filtrar sólo aquellos clientes con más de un contrato. Esta columna será un número creciente sobre el "grupo o ventana" utilizando el comando row_number:

# Definimos una ventana por la que hacer una partición, la cual tiene que estar obligatoriamente ordenada además por alguna de las columnas. En nuestro caso, como queremos saber cuántos id_cliente tenemos repetidos (si hay alguno repetido ya significa que dicho cliente tiene más de un contrato), agrupamos por la columna "id_cliente" y ordenamos por ejemplo por la columna "importe" en orden descendente, para que nos muestre el primero el contrato más caro que el cliente tiene con nuestra compañía.

# Importamos la función Window del paquete pyspark.sql.window y la definimos
from pyspark.sql.window import Window
window_cliente = Window.partitionBy("id_cliente").orderBy(desc("importe"))

df_facturas_mes_ant = df_facturas_mes_ant.withColumn("count_contratos", row_number().over(window_cliente))
# Como puede haber ventanas con más de 2 elementos (por ejemplo el caso del id_cliente 1), creamos otra columna ("total_contratos") que nos informe del número maximo de la columna que acabamos de crear ("count_contratos").
df_facturas_mes_ant = df_facturas_mes_ant.withColumn("total_contratos", max("count_contratos").over(window_cliente.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)))

df_facturas_mes_ant.show(n=40)

+----------+---------+-------+----------+-----------------+---------------+---------------+
|id_cliente|id_oferta|importe|     fecha|           nombre|count_contratos|total_contratos|
+----------+---------+-------+----------+-----------------+---------------+---------------+
|         1|        6|  124.5|2020-07-31|      Pablo Perez|              1|              3|
|         1|        6|  118.6|2020-07-31|      Pablo Perez|              2|              3|
|         1|        9|   36.5|2020-07-31|      Pablo Perez|              3|              3|
|         2|        4|   95.0|2020-07-31|  Eduardo Redondo|              1|              1|
|         3|        5|  102.5|2020-07-31|  Roberto Salazar|              1|              1|
|         5|        7|   80.5|2020-07-31|       Ana Robles|              1|              2|
|         5|        9|   40.5|2020-07-31|       Ana Robles|              2|              2|
|         6|        8|   84.0|2020-07-31|     David Roldan|              1|     

In [74]:
# Finalmente, ya podemos filtrar aquellas filas en las que coincidan count_contratos y total_contratos, y que además count_contratos sea mayor que 1 para obtener aquellos clientes del mes anterior que tenían más de un contrato:
df_clientes_vip = df_facturas_mes_ant.filter((col("count_contratos") == col("total_contratos")) & (col("count_contratos") > 1))
df_clientes_vip.show()
# Nota: Muy importante cuando quiera poner múltiples condiciones en un filter, que cada condición vaya entre paréntesis.

+----------+---------+-------+----------+-----------------+---------------+---------------+
|id_cliente|id_oferta|importe|     fecha|           nombre|count_contratos|total_contratos|
+----------+---------+-------+----------+-----------------+---------------+---------------+
|         1|        9|   36.5|2020-07-31|      Pablo Perez|              3|              3|
|         5|        9|   40.5|2020-07-31|       Ana Robles|              2|              2|
|        15|        2|   78.5|2020-07-31|Cristian Cuadrado|              2|              2|
|        18|        9|   37.5|2020-07-31|       Laura Luiz|              2|              2|
|        22|        7|   69.5|2020-07-31|     Clara Suarez|              2|              2|
|        25|       10|  31.99|2020-07-31|     Ines Barcero|              2|              2|
|        27|        9|   41.5|2020-07-31|   Roberta Varado|              2|              2|
|        30|        2|   70.0|2020-07-31|    Marta Rodrigo|              2|     

**9. Crear columnas para los DF de facturas del mes actual y el anterior llamadas: "fibra_300MB", "movil_llamadas_ilimitadas", "television", asignando un 1 si la oferta del cliente contiene el elemento correspondiente y un 0 en caso contrario.** (Ej. Como el id_oferta 1 tiene Fibra Optica 300 MB, todos los clientes que tengan contratada la oferta 1 deberán tener el valor 1 en la columna fibra_300MB) (contains, withColumnRenamed)

In [ ]:
df_facturas_mes_ant = df_facturas_mes_ant.drop('nombre','count_contratos','total_contratos')
#df_facturas_mes_ant.show(4)
#df_facturas_mes_actual.show(4)
df_ofertas.show(3)
df_facturas_mes_ant.show(3)


In [77]:
# En primer lugar, es necesario agregar la descripción del id_oferta a partir del DF df_ofertas.

# OJO ! Hemos de tener cuidado porque hay una columna que tiene el mismo nombre en ambos DF (importe), y que además no forma parte de la clave de columnas por las que se realiza el join (["id_oferta"] en este caso). Es necesario que renombremos una de ellas, puesto que si no habrá ambigüedad y obtendremos un error.

# Renombramos la columna "importe" del DF df_ofertas a "precio_oferta" para evitar el conflicto:
df_facturas_mes_ant_with_ofertas = df_facturas_mes_ant.join(df_ofertas.withColumnRenamed("importe","precio_oferta"),["id_oferta"],"left") \
    .withColumn("fibra_300MB", when(col("descripcion").contains("Fibra Optica 300MB"),lit(1)).otherwise(lit(0))) \
    .withColumn("movil_llamadas_ilimitadas", when((col("descripcion").contains("Movil llamadas ilimitadas")) | 
                                                  (col("descripcion").contains("Movil llamadas y datos ilimitados")),lit(1)).otherwise(lit(0))) \
    .withColumn("television", when(col("descripcion").contains("Television"),lit(1)).otherwise(lit(0))) \
    .sort("id_cliente")

# Cuando vemos que nuestro DF tiene un número de columnas elevado, de forma que con el comando show ya no entran todas en una misma linea, es preferible usar el comando display para mostrar el DF (sólamente funcionará en Databricks).
#display(df_facturas_mes_ant_with_ofertas)
df_facturas_mes_ant_with_ofertas.show()

+---------+----------+-------+----------+-----------------+---------------+---------------+-----------------+--------------------+-------------+-----------+-------------------------+----------+
|id_oferta|id_cliente|importe|     fecha|           nombre|count_contratos|total_contratos|           nombre|         descripcion|precio_oferta|fibra_300MB|movil_llamadas_ilimitadas|television|
+---------+----------+-------+----------+-----------------+---------------+---------------+-----------------+--------------------+-------------+-----------+-------------------------+----------+
|        9|         1|   36.5|2020-07-31|      Pablo Perez|              3|              3|      Pablo Perez|Fijo llamadas ili...|         42.5|          0|                        1|         0|
|        6|         1|  118.6|2020-07-31|      Pablo Perez|              2|              3|      Pablo Perez|Fibra Optica 600M...|        124.5|          0|                        1|         1|
|        6|         1|  124.5|

**10. Identificar si el importe de la factura del mes actual es igual que el de la oferta estándar correspondiente, creando una nueva variable binaria (valores 0 - 1) llamada "oferta_no_estandar", que valdrá 1 si al cliente se le ha realizado una oferta especial por ejemplo por cambiar de compañía** (when col < col)

In [78]:
#df_facturas_mes_actual.show(5)
#df_ofertas.show(5)

df_ofertas = df_ofertas.withColumnRenamed("importe","importe_oferta")
df_new =df_facturas_mes_actual.join(df_ofertas,["id_oferta"],"left").orderBy("id_oferta").withColumn("oferta_no_estandar",when(col("importe") != col("importe_oferta"),lit(1)))\
.na.fill(value=0,subset = "oferta_no_estandar").show()




+---------+----------+-------+----------+--------------------+--------------+------------------+
|id_oferta|id_cliente|importe|     fecha|         descripcion|importe_oferta|oferta_no_estandar|
+---------+----------+-------+----------+--------------------+--------------+------------------+
|        1|        30|   70.0|2020-08-31|Fibra Optica 300M...|          70.0|                 0|
|        1|        12|   70.0|2020-08-31|Fibra Optica 300M...|          70.0|                 0|
|        1|         4|   70.0|2020-08-31|Fibra Optica 300M...|          70.0|                 0|
|        2|        15|   78.5|2020-08-31|Fibra Optica 600M...|          78.5|                 0|
|        2|        30|   71.0|2020-08-31|Fibra Optica 600M...|          78.5|                 1|
|        2|        22|   78.5|2020-08-31|Fibra Optica 600M...|          78.5|                 0|
|        3|        19|   85.0|2020-08-31|Fibra Optica 300M...|          85.0|                 0|
|        3|        23|   85.0|

In [79]:
#OTRA FORMA
# Podemos reaprovechar el DataFrame generado en el ejercicio anterior:
df_facturas_mes_ant_with_ofertas = df_facturas_mes_ant_with_ofertas.withColumn("oferta_no_estandar", when(col("importe") < col("precio_oferta"), lit(1)).otherwise(lit(0)))
#display(df_facturas_mes_ant_with_ofertas)
df_facturas_mes_ant_with_ofertas.show()

+---------+----------+-------+----------+-----------------+---------------+---------------+-----------------+--------------------+-------------+-----------+-------------------------+----------+------------------+
|id_oferta|id_cliente|importe|     fecha|           nombre|count_contratos|total_contratos|           nombre|         descripcion|precio_oferta|fibra_300MB|movil_llamadas_ilimitadas|television|oferta_no_estandar|
+---------+----------+-------+----------+-----------------+---------------+---------------+-----------------+--------------------+-------------+-----------+-------------------------+----------+------------------+
|        9|         1|   36.5|2020-07-31|      Pablo Perez|              3|              3|      Pablo Perez|Fijo llamadas ili...|         42.5|          0|                        1|         0|                 1|
|        6|         1|  118.6|2020-07-31|      Pablo Perez|              2|              3|      Pablo Perez|Fibra Optica 600M...|        124.5|    

**11. Crea una columna llamada "importe_total" que obtenga a finales del mes_actual el importe total (suma de importes de todos los contratos de cada cliente). Para los clientes que se han dado de baja este mes y estaban en el anterior, esta columna representará el último importe que pagaron a finales del mes pasado.** (drop, full join, isNull, coalesce)

In [ ]:
# En primer lugar, hacemos un display de los 2 dataframes de facturas para ver qué campos tiene cada uno, ya que hemos ido asignando distintos objetos dataframes a la referencia donde apuntan las variables "df_facturas_mes_ant" y "df_facturas_mes_actual", sobreescribiendo los que había anteriormente.

In [ ]:
#display(df_facturas_mes_actual)

In [ ]:
#display(df_facturas_mes_ant)

In [80]:
# Renombramos los campos que no queremos porque nos van a provocar conflicto (no podemos eliminarlos porque hay clientes que existian en el mes anterior y no se han dado de baja este mes y viceversa, y por tanto si eliminásemos campos, perderíamos información en el cruce).

# Además, debemos escoger cuidadosamente la clave de unión porque al haber clientes con más de un contrato, si no la elegimos correctamente, obtendremos más registros de los que buscamos. Esto se debe a que hay clientes que tienen contratada la misma oferta 2 veces, como es el caso del cliente con id = 1, que tiene contratada la oferta con id = 6, 2 veces.

# Vamos a hacerlo **MAL** la primera vez para observar lo que comentamos con el cliente con id = 1 de que se generan más registros:

# El tipo de join que buscamos es un "full join" porque queremos obtener tanto los contratos que estaban en el mes anterior y no en el actual, como los que están en los dos meses, como los que están en el mes actual pero no estaban en el anterior. 
df_facturas_mes_actual_renamed = df_facturas_mes_actual.withColumnRenamed("edad","edad_actual") \
                                                       .withColumnRenamed("grupo_edad","grupo_edad_actual") \
                                                       .withColumnRenamed("fecha","fecha_actual") \
                                                       .withColumnRenamed("importe","importe_actual") \

df_facturas = df_facturas_mes_actual_renamed.join(df_facturas_mes_ant,["id_cliente","id_oferta"],"full").orderBy("id_cliente")
df_facturas.show()
#display(df_facturas)
# Si observamos las tablas del mes anterior y el actual mostradas en las celdas inmediatamente anteriores, vemos que el cliente con id = 1 tenía en el mes anterior 3 contratos y en el mes actual se ha dado de baja del contrato con id_oferta 9.

# Sin embargo, como para la clave del join hemos escogido sólo los campos (id_cliente, id_oferta) y los dos primeros contratos tienen el mismo valor para estos dos campos, se duplican los resultados, puesto que deberíamos deberíamos obtener para el cliente con id = 1, 3 filas en la tabla final, y estamos obteniendo 5.  

+----------+---------+--------------+------------+-------+----------+----------------+---------------+---------------+----------------+
|id_cliente|id_oferta|importe_actual|fecha_actual|importe|     fecha|          nombre|count_contratos|total_contratos|          nombre|
+----------+---------+--------------+------------+-------+----------+----------------+---------------+---------------+----------------+
|         1|        6|         118.6|  2020-08-31|  124.5|2020-07-31|     Pablo Perez|              1|              3|     Pablo Perez|
|         1|        6|         124.5|  2020-08-31|  124.5|2020-07-31|     Pablo Perez|              1|              3|     Pablo Perez|
|         1|        6|         118.6|  2020-08-31|  118.6|2020-07-31|     Pablo Perez|              2|              3|     Pablo Perez|
|         1|        6|         124.5|  2020-08-31|  118.6|2020-07-31|     Pablo Perez|              2|              3|     Pablo Perez|
|         1|        9|          null|        nul

In [83]:
# Para solucionar este problema, tenemos que encontramos una clave de join incluyendo algún campo más que haga que los dos primeros contratos del cliente con id = 1 no tengan los mismos valores para la clave del join. Una opción es replicar la columna "count_contratos" en la tabla del mes actual, que ya existe en el mes anterior y de esta forma conseguimos que en la clave de particionamiento el contrato 1 del cliente 1 tenga los valores (1,6,1), mientras que el segundo tendrá valores (1,6,2), lo que hará que ya no se dupliquen registros
winSpec_cliente_oferta = Window.partitionBy("id_cliente","id_oferta").orderBy(desc("importe_actual"))
df_facturas_mes_actual_renamed = df_facturas_mes_actual_renamed.withColumn("count_contratos", row_number().over(winSpec_cliente_oferta))

df_facturas = df_facturas_mes_actual_renamed.join(df_facturas_mes_ant,["id_cliente","id_oferta","count_contratos"],"full").orderBy("id_cliente")

In [84]:
# Reducimos el numero de columnas eliminando todas aquellas con el sufijo "_actual". Sin embargo, antes de eliminarlas, debemos dejar completamente relleno el campo edad, ya que para los clientes nuevos del mes actual, el valor para este campo será "null", puesto que viene sólo informado en el campo "edad_actual" ya que en el mes_anterior el cliente no existía y por tanto no ha habido match al hacer el join. 
df_facturas = df_facturas.withColumn("edad", when(col("edad").isNull(), col("edad_actual")).otherwise(col("edad"))) \
                         .withColumn("grupo_edad", coalesce("grupo_edad","grupo_edad_actual"))

# El comando coalesce coge de las columnas especificadas entre paréntesis, la primera que no sea "null", comenzando por la de más a la izquierda.
#display(df_facturas)

AnalysisException: cannot resolve 'edad' given input columns: [count_contratos, fecha, fecha_actual, id_cliente, id_oferta, importe, importe_actual, nombre, nombre, total_contratos];
'Project [id_cliente#2022L, id_oferta#2023L, count_contratos#2024, importe_actual#1935, fecha_actual#1930, importe#83, fecha#84, nombre#19, total_contratos#1562, nombre#1633, CASE WHEN isnull('edad) THEN 'edad_actual ELSE 'edad END AS edad#2035]
+- Sort [id_cliente#2022L ASC NULLS FIRST], true
   +- Project [coalesce(id_cliente#89L, id_cliente#81L) AS id_cliente#2022L, coalesce(id_oferta#90L, id_oferta#82L) AS id_oferta#2023L, coalesce(count_contratos#2016, count_contratos#1553) AS count_contratos#2024, importe_actual#1935, fecha_actual#1930, importe#83, fecha#84, nombre#19, total_contratos#1562, nombre#1633]
      +- Join FullOuter, (((id_cliente#89L = id_cliente#81L) AND (id_oferta#90L = id_oferta#82L)) AND (count_contratos#2016 = count_contratos#1553))
         :- Project [id_cliente#89L, id_oferta#90L,

In [85]:
# Ahora ya sí que podemos eliminar los campos con sufijo "_actual" excepto importe_actual, así como de los campos (fecha,nombre,count_contratos,total_contratos,id_oferta) mediante el comando drop.
df_facturas = df_facturas.drop("edad_actual","fecha_actual","grupo_edad_actual","nombre","fecha","fecha_actual","count_contratos","total_contratos","id_oferta")
#display(df_facturas)

In [86]:
# Por ultimo, calculamos dos nuevas columnas auxiliares, una con la suma del importe para cada cliente el mes actual y otra semejante pero para el mes anterior, para finalmente obtener a partir de ellas la columna buscada en el enunciado "importe_total"
winSpec_cliente = Window.partitionBy("id_cliente").orderBy(desc("importe_actual"))
df_importe_total = df_facturas.withColumn("suma_cliente_mes_actual", sum("importe_actual").over(winSpec_cliente.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))) \
                              .withColumn("suma_cliente_mes_ant", sum("importe").over(winSpec_cliente.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))) \

df_importe_total.show()
#display(df_importe_total)

+----------+--------------+-------+-----------------------+--------------------+
|id_cliente|importe_actual|importe|suma_cliente_mes_actual|suma_cliente_mes_ant|
+----------+--------------+-------+-----------------------+--------------------+
|         1|         124.5|  124.5|                  243.1|               279.6|
|         1|         118.6|  118.6|                  243.1|               279.6|
|         1|          null|   36.5|                  243.1|               279.6|
|         2|          95.0|   95.0|                   95.0|                95.0|
|         3|         102.5|  102.5|                  102.5|               102.5|
|         4|          70.0|   null|                   70.0|                null|
|         5|          80.5|   80.5|                  121.0|               121.0|
|         5|          40.5|   null|                  121.0|               121.0|
|         5|          null|   40.5|                  121.0|               121.0|
|         6|          84.0| 

In [104]:
# Observamos que en las columnas "suma" calculadas, tenemos algunos valores a "null" debido a que el contrato o bien no existía en el mes anterior, o bien se ha dado de baja en el mes actual. Por tanto, como dice el enunciado, para los clientes dados de baja este mes (tendrán el campo "suma_cliente_mes_actual" a "null"), se cogerá como importe total el que tenian a finales del mes anterior la suma de sus contratos con la compañía.

# Por último, dejamos un único registro por cliente
df_importe_total = df_importe_total.withColumn("importe_total", coalesce("suma_cliente_mes_actual","suma_cliente_mes_ant")) \
                                   .drop("importe_actual","importe","suma_cliente_mes_actual","suma_cliente_mes_ant") \
                                   .dropDuplicates(["id_cliente"])

#display(df_importe_total)

AnalysisException: cannot resolve 'suma_cliente_mes_actual' given input columns: [id_cliente, importe_total];
'Project [id_cliente#2022L, coalesce('suma_cliente_mes_actual) AS importe_total#2757]
+- Deduplicate [id_cliente#2022L]
   +- Project [id_cliente#2022L, importe_total#2075]
      +- Project [id_cliente#2022L, importe_actual#1935, importe#83, suma_cliente_mes_actual#2040, suma_cliente_mes_ant#2046, coalesce(suma_cliente_mes_actual#2040, suma_cliente_mes_ant#2046) AS importe_total#2075]
         +- Project [id_cliente#2022L, importe_actual#1935, importe#83, suma_cliente_mes_actual#2040, suma_cliente_mes_ant#2046]
            +- Project [id_cliente#2022L, importe_actual#1935, importe#83, suma_cliente_mes_actual#2040, suma_cliente_mes_ant#2046, suma_cliente_mes_ant#2046]
               +- Window [sum(importe#83) windowspecdefinition(id_cliente#2022L, importe_actual#1935 DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) AS suma_cliente_me

d **12. Calcular por separado 4 DataFrames con informacion del mes anterior: Cada DataFrame se realizará agrupando la información por cada "grupo_edad" diferente:**
- Una columna con la suma de los importes de todos los contratos de clientes que pertenezcan al "grupo_edad" 1 (idem para el grupo_edad 2, para el 3 y para el 4).
- Otra columna con el número de contratos que hay en cada grupo,
- Otra columna con el número de clientes que hay en cada grupo,
- Una tercera columna con la media de gasto, por persona según el grupo de edad al que pertenecen. <br>
**Obtendremos por tanto 4 DataFrames con 1 fila cada uno y 4 columnas (grupo_edad, importe_total, n_clientes, gasto_medio_cliente) y que finalmente uniremos en uno único.** (countDistinct, union, unionByName, cache, cast(DecimalType(precision,scale))) </br>

In [ ]:
#display(df_facturas_mes_ant)

In [94]:
df_grupo_edad_1 = df_new_facturas_mes_ant.filter(col("grupo_edad") == 1) \
                                     .groupBy("grupo_edad").agg(sum("importe").alias("importe_cliente"),
                                                                count("id_cliente").alias("n_contratos"),
                                                                countDistinct("id_cliente").alias("n_clientes"))

# Se utiliza countDistinct porque si solo hacemos un count obtenemos el número de contratos de los clientes del grupo_edad 1, pero como hemos visto, es posible que un mismo cliente tenga más de un contrato con la compañía y por tanto es necesario que si se da este caso sólo me cuente una vez cada cliente.
#display(df_grupo_edad_1)
df_grupo_edad_1.show()

+----------+---------------+-----------+----------+
|grupo_edad|importe_cliente|n_contratos|n_clientes|
+----------+---------------+-----------+----------+
|         1|          502.0|          6|         5|
+----------+---------------+-----------+----------+



In [98]:
# Los resultados nos muestran que el grupo_edad 1 (18-25 años) aporta 502€ a la compañía según la muestra extraída, aunque este dato no tiene mucha relevancia, ya que en la muestra recogida el número de clientes del grupo_edad 1 no es el mismo que el grupo_edad 2,3 o 4, por lo que lo relevante es obtener el gasto medio de un cliente perteneciente a este grupo   
df_grupo_edad_1 = df_grupo_edad_1.withColumn("gasto_medio_cliente", col("importe_cliente") / col("n_clientes"))                                                             

df_grupo_edad_1.show()

+----------+---------------+-----------+----------+-------------------+
|grupo_edad|importe_cliente|n_contratos|n_clientes|gasto_medio_cliente|
+----------+---------------+-----------+----------+-------------------+
|         1|          502.0|          6|         5|              100.4|
+----------+---------------+-----------+----------+-------------------+



In [100]:
# Realizamos estas mismas operaciones para los grupo_edad restantes
df_grupo_edad_2 = df_new_facturas_mes_ant.filter(col("grupo_edad") == 2) \
                                     .groupBy("grupo_edad").agg(sum("importe").alias("importe_cliente"),
                                                                count("id_cliente").alias("n_contratos"),
                                                                countDistinct("id_cliente").alias("n_clientes")) \
                                     .withColumn("gasto_medio_cliente", col("importe_cliente") / col("n_clientes"))


df_grupo_edad_3 = df_new_facturas_mes_ant.filter(col("grupo_edad") == 3) \
                                     .groupBy("grupo_edad").agg(sum("importe").alias("importe_cliente"),
                                                                count("id_cliente").alias("n_contratos"),
                                                                countDistinct("id_cliente").alias("n_clientes")) \
                                     .withColumn("gasto_medio_cliente", col("importe_cliente") / col("n_clientes"))

df_grupo_edad_4 = df_new_facturas_mes_ant.filter(col("grupo_edad") == 4) \
                                     .groupBy("grupo_edad").agg(sum("importe").alias("importe_cliente"),
                                                                count("id_cliente").alias("n_contratos"),
                                                                countDistinct("id_cliente").alias("n_clientes")) \
                                     .withColumn("gasto_medio_cliente", col("importe_cliente") / col("n_clientes"))

# Por último, procedemos a unir los 4 dataframes generados en uno. Para ello, podemos ejecutar 2 comandos:
#     1. union --> para que este comando funcione, es necesario que TODOS los DataFrames a unir tengan el MISMO número de columnas y el MISMO orden de columnas.
#     2. unionByName --> para que este comando funcione, es necesario que TODOS los DataFrames a unir tengan el MISMO número de columnas, pero los campos pueden estar desordenados. 
df_grupo_edad_resumen = df_grupo_edad_1.unionByName(df_grupo_edad_2).unionByName(df_grupo_edad_3).unionByName(df_grupo_edad_4).cache()

#display(df_grupo_edad_resumen)
df_grupo_edad_resumen.show()

+----------+---------------+-----------+----------+-------------------+
|grupo_edad|importe_cliente|n_contratos|n_clientes|gasto_medio_cliente|
+----------+---------------+-----------+----------+-------------------+
|         1|          502.0|          6|         5|              100.4|
|         2|         645.59|          8|         5|            129.118|
|         3|         1037.5|         13|         9| 115.27777777777777|
|         4|          566.5|          7|         6|  94.41666666666667|
+----------+---------------+-----------+----------+-------------------+



In [101]:
# Por ultimo, vamos a castear el campo "gasto_medio_cliente" para obtener el resultado solo con 2 decimales
from pyspark.sql.types import *
df_grupo_edad_resumen = df_grupo_edad_resumen.withColumn("gasto_medio_cliente", col("gasto_medio_cliente").cast(DecimalType(17,2)))

#display(df_grupo_edad_resumen)
df_grupo_edad_resumen.show()

+----------+---------------+-----------+----------+-------------------+
|grupo_edad|importe_cliente|n_contratos|n_clientes|gasto_medio_cliente|
+----------+---------------+-----------+----------+-------------------+
|         1|          502.0|          6|         5|             100.40|
|         2|         645.59|          8|         5|             129.12|
|         3|         1037.5|         13|         9|             115.28|
|         4|          566.5|          7|         6|              94.42|
+----------+---------------+-----------+----------+-------------------+



**Como conclusión de este último apartado 12, podemos decir que:**
- El grupo de edad 1 (18-25 años) pese a gozar de un amplio conocimiento tecnológico y necesidades de consumo, no tienen poder adquisitivo suficiente.
- El grupo de edad 2 (26-40 años) es el que más gasta en este tipo de servicios, lo cual es un resultado coherente ya que cuentan tanto con cierto poder adquisitivo, como con cierto conocimiento de las nuevas tecnologías.
- El grupo de edad 3 (41-65 años) es seguramente el que mayor poder adquisitivo tenga, pero su conocimiento de la tecnología es por lo general más reducido, no despertando tanto interés en ellos todo este tipo de servicios. Pese a ello, son el segundo grupo que más gasta.
- El grupo de edad 4 (> 65 años) es el que menor gasto realiza en nuestra compañía, seguramente por no estar tan interesados en las nuevas tecnologías.